In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline


In [2]:
%pwd
import sys
sys.path.append("../../../fastai/fastai/")
print(sys.path)


['', '/home/rich/git/fastai/courses/ml1', '/home/rich/anaconda3/lib/python37.zip', '/home/rich/anaconda3/lib/python3.7', '/home/rich/anaconda3/lib/python3.7/lib-dynload', '/home/rich/anaconda3/lib/python3.7/site-packages', '/home/rich/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/home/rich/.ipython', '../../../fastai/fastai/']


In [3]:
#PATH = "~/git/fastai"
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

/home/rich/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
#now, since we're usign scikit-learn, we must convert categorical columns into dummy variables
def dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked', 'Ticket_Lett', 'Cabin_Letter', 'Name_Title', 'Fam_Size']):
    for column in columns:
       train[column] = train[column].apply(lambda x: str(x))
       test[column] = test[column].apply(lambda x: str(x))
        
       good_cols = [column+'_'+i for i in train[column].unique() if i in test[column].unique()]
        
       train = pd.concat((train, pd.get_dummies(train[column], prefix = column)[good_cols]), axis = 1)
       test = pd.concat((test, pd.get_dummies(test[column], prefix = column)[good_cols]), axis = 1)
        
       del train[column]
       del test[column]
    return train,test

def split_vals(a,n): return a[:n], a[n:]


In [5]:
PATH = "~/git/fastai/data/human_numbers/"

train = pd.read_csv(f'{PATH}train.csv', low_memory=False)
test = pd.read_csv(f'{PATH}test.csv', low_memory=False)


    DATASET PREPROCESSING

In [ ]:
#to try -- secondary column indicating rank of each pixel value 1-5

        HYPERPARAMETER TUNING

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_features='auto', oob_score=True, random_state=1,n_jobs=-1)

param_grid = {"criterion" : ["gini", "entropy"], "min_samples_leaf" : [1,5,10],
             "min_samples_split": [2,4,10,12,16],"n_estimators":[50,100,400,700,1000]}



gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=3,n_jobs=-1)

gs

#gs = gs.fit(train.iloc[:,1:], train.iloc[:,0])

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=True, random_state=1, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'criterion': ['gini', 'entropy'], 'min_samples_leaf': [1, 5, 10], 'min_samples_split': [2, 4, 10, 12, 16], 'n_estimators': [50, 100, 400, 700, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [ ]:
gs = gs.fit(train.iloc[:,1:], train.iloc[:,0])

In [11]:
#trying this from brian farris solution
from sklearn import cross_validation
from sklearn.metrics import accuracy_score

trainbf = pd.read_csv(f'{PATH}train.csv', low_memory=False)
features = train.columns[1:]
X = train[features]
y = train['label']

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X/255.,y,test_size=0.1,random_state=0)

clf_rf = RandomForestClassifier()
#fit the classifier to the datasets
clf_rf.fit(X_train, y_train)

#predict using test dataset
y_pred_rf = clf_rf.predict(X_test)

#calc accuracy
acc_rf = accuracy_score(y_test, y_pred_rf)

print("Random Forest Accuracy: ", str(acc_rf))


Random Forest Accuracy:  0.9364285714285714
